In [275]:
from pandas import read_csv, DataFrame
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.stats as stats

dataset = read_csv('Ar_300_rough_collision.dat', sep=' ')
dataset.head()

,Number,Time,Temperature,Uvx,Uvy,Uvz,Uex,Uey,Uez
0,1,175,303.696396,-4.658513,-5.898589,-3.567033,2.745380,-1.853520,4.392539
1,2,182,307.713569,-0.799739,-0.019740,-3.964088,0.730010,-4.444037,2.841238
2,3,195,312.573645,-1.971708,3.424431,-3.939806,1.070360,-2.532539,2.700840
3,4,26,309.180417,2.485977,-0.693671,-6.412166,1.574913,0.404154,4.182970
4,5,240,299.588882,-0.079111,-6.650128,-3.855634,1.709926,-0.126030,6.029875


In [274]:
#find absolute value of z-score for each observation
z = np.abs(stats.zscore(dataset))

#only keep rows in dataframe with all z-scores less than absolute value of 3 
data_clean = dataset[(z<3).all(axis=1)]

#find how many rows are left in the dataframe 
dataset = data_clean
data_clean.shape

(95562, 9)

In [276]:
Q1 = dataset.quantile(q=0.25)
Q3 = dataset.quantile(q=0.75)
IQR = dataset.apply(stats.iqr)

# Оставляем только строки в DataFrame, значения которых находятся в пределах 1.5*IQR от Q1 и Q3
data_clean = dataset[~((dataset < (Q1 - 1.5 * IQR)) | (dataset > (Q3 + 1.5 * IQR))).any(axis=1)]

# Находим количество оставшихся строк в DataFrame
data_clean.shape

(87925, 9)

In [162]:
dataset.corr()

,Number,Time,Temperature,Uvx,Uvy,Uvz,Uex,Uey,Uez,Collision,Gas
Number,1.000000,-0.001521,-0.001252,-0.004065,0.000002,-0.001100,-0.002802,0.000441,-0.000235,0.000000,0.000000
Time,-0.001521,1.000000,-0.001589,0.000897,0.001722,0.275380,0.000750,0.002323,-0.275415,-0.033025,-0.140247
Temperature,-0.001252,-0.001589,1.000000,-0.001160,-0.000047,0.005853,-0.000652,-0.000135,-0.008024,0.010990,-0.009690
Uvx,-0.004065,0.000897,-0.001160,1.000000,0.001857,0.002938,0.788465,-0.023988,-0.002152,-0.000007,0.001315
Uvy,0.000002,0.001722,-0.000047,0.001857,1.000000,-0.000511,-0.024228,0.802677,0.000043,0.000001,0.000898
Uvz,-0.001100,0.275380,0.005853,0.002938,-0.000511,1.000000,0.001459,0.001223,-0.870207,-0.000006,-0.662364
Uex,-0.002802,0.000750,-0.000652,0.788465,-0.024228,0.001459,1.000000,0.001262,-0.001254,-0.000398,0.002277
Uey,0.000441,0.002323,-0.000135,-0.023988,0.802677,0.001223,0.001262,1.000000,-0.000476,0.001865,-0.000789
Uez,-0.000235,-0.275415,-0.008024,-0.002152,0.000043,-0.870207,-0.001254,-0.000476,1.000000,-0.000684,0.663057
Collision,0.000000,-0.033025,0.010990,-0.000007,0.000001,-0.000006,-0.000398,0.001865,-0.000684,1.000000,0.000000


In [209]:
dataset = dataset.query('Time < 1001')
dataset = dataset.query('Temperature < 401')

In [182]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386712 entries, 0 to 399995
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Number       386712 non-null  int64  
 1   Time         386712 non-null  int64  
 2   Temperature  386712 non-null  float64
 3   Uvx          386712 non-null  float64
 4   Uvy          386712 non-null  float64
 5   Uvz          386712 non-null  float64
 6   Uex          386712 non-null  float64
 7   Uey          386712 non-null  float64
 8   Uez          386712 non-null  float64
 9   Collision    386712 non-null  int64  
 10  Gas          386712 non-null  int64  
dtypes: float64(7), int64(4)
memory usage: 35.4 MB


In [281]:
trg = dataset.drop(['Number', 'Time'], axis=1)[['Uex','Uey', 'Uez']]
trn = dataset.drop(['Number', 'Uex','Uey', 'Uez'], axis=1)

In [282]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(trn, trg, test_size=0.3)

In [305]:
model = RandomForestRegressor(n_estimators=200, verbose=2, n_jobs=8)
# model = RandomForestRegressor(n_estimators=200, verbose=2, n_jobs=8, max_depth=10, min_samples_split=2, min_impurity_decrease=3)
#{'criterion': 'squared_error', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 200}

In [306]:
model.fit(Xtrn, Ytrn)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 200building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200

building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    2.4s


building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66 of 200
building tree 67 of 200
building tree 68 of 200
building tree 69 of 200
building tree 70 of 200
building tree 71 of 200
building tree 72 of 200
building tree 73 of 200
building tree 74 of 200
building tree 75 of 200
building tree 76 of 200
building tree 77

[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   13.1s


building tree 157 of 200
building tree 158 of 200
building tree 159 of 200
building tree 160 of 200
building tree 161 of 200
building tree 162 of 200
building tree 163 of 200
building tree 164 of 200
building tree 165 of 200
building tree 166 of 200
building tree 167 of 200
building tree 168 of 200
building tree 169 of 200
building tree 170 of 200
building tree 171 of 200
building tree 172 of 200
building tree 173 of 200
building tree 174 of 200
building tree 175 of 200
building tree 176 of 200
building tree 177 of 200
building tree 178 of 200
building tree 179 of 200
building tree 180 of 200
building tree 181 of 200
building tree 182 of 200
building tree 183 of 200
building tree 184 of 200
building tree 185 of 200
building tree 186 of 200
building tree 187 of 200
building tree 188 of 200
building tree 189 of 200
building tree 190 of 200
building tree 191 of 200
building tree 192 of 200
building tree 193 of 200
building tree 194 of 200
building tree 195 of 200
building tree 196 of 200


[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:   17.9s finished


RandomForestRegressor(n_estimators=200, n_jobs=8, verbose=2)

In [285]:
from joblib import dump, load
dump(model, 'Ar_300_rough_collision.joblib')

['Ar_300_rough_collision.joblib']

In [ ]:
# model = load('model.joblib')

In [291]:
Xtest

,Time,Temperature,Uvx,Uvy,Uvz
48039,91,297.791348,-2.918791,0.450256,-0.987129
54746,566,304.337401,0.143628,-3.945810,-0.589029
81875,800,294.841545,2.389882,2.571588,-2.594221
98811,20,304.940162,1.466171,-4.718521,-5.728716
66082,843,319.972254,1.713699,0.401296,-3.526438
...,...,...,...,...,...
50629,88,289.912803,1.286908,1.343710,-3.438700
77786,22,309.431664,-0.009635,2.887370,-3.125996
68533,1624,312.872073,-0.807877,-1.647793,-4.900831
70111,242,303.014087,-5.560472,-5.574286,-2.416310


In [307]:
test_prediction = model.predict(Xtest)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.5s finished


In [303]:
test_prediction

array([[-2.95406177e-01,  3.24875827e-02,  2.76382237e+00],
       [ 1.59521357e-02, -1.04662480e-01,  2.79079196e+00],
       [-1.08970129e-02,  6.42675587e-02,  2.78679418e+00],
       ...,
       [-7.86995600e-02,  3.23788661e-04,  2.75500517e+00],
       [-5.99613293e-02,  6.16373679e-02,  2.80175873e+00],
       [-1.15120068e-03, -3.54753315e-03,  2.77038154e+00]])

In [308]:
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import numpy as np
MAE = mean_absolute_error(Ytest, test_prediction)
MSE = mean_squared_error(Ytest, test_prediction)
RMSE = np.sqrt(MSE)
r2 = r2_score(Ytest, test_prediction)
for key, value in {'MAE': MAE, 'MSE': MSE, 'RMSE': RMSE, 'r2': r2}.items():
    print(key, value)

MAE 1.5764239588308167
MSE 4.223607179052316
RMSE 2.0551416445229065
r2 0.18445144759729992


In [292]:
dictionary = dict(zip(['Time', 'Temperature', 'Uvx', 'Uvy', 'Uvz'], [[175],[303.696395975584],[-4.65851294521058],[-5.89858912205267],[-3.56703257911055]]))
array = DataFrame(dictionary)
array

,Time,Temperature,Uvx,Uvy,Uvz
0,175,303.696396,-4.658513,-5.898589,-3.567033


In [293]:
model.predict(array)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


array([[ 2.16272898, -1.17168125,  4.07667576]])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Ваши вектора
vector1 = [1, 2, 3]
vector2 = [4, 5, 6]

# Создание нового рисунка
fig = plt.figure()

# Добавление 3D-осей
ax = fig.add_subplot(111, projection='3d')

# Построение векторов
ax.quiver(0, 0, 0, vector1[0], vector1[1], vector1[2], color='r')
ax.quiver(0, 0, 0, vector2[0], vector2[1], vector2[2], color='b')

# Установка пределов для осей
ax.set_xlim([0, max(vector1[0], vector2[0])])
ax.set_ylim([0, max(vector1[1], vector2[1])])
ax.set_zlim([0, max(vector1[2], vector2[2])])

# Показать график
plt.show()
